## Gathering Data From Basketball Reference

In this notebook, I will build a number of functions to automate the process of collecting data from basketball-reference.com. These functions will take in a url that corresponds to a team or individual player. They will scrape, clean, and arrange the data in a way that it can be used in a machine learning model. This notebook is going to use the 2016-2017 season as an example.

In [5]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup  #the webscraping module
import requests                #for accessing the site to be scraped
import time

## Collecting Team Stats

For each player in the dataset, we want to have the basic and advanced statistics for their opponent on a given day.  In order to do that we need to create a list to hold the web address for each team's basic and advanced statistics.  For example, the web address for the page that holds the basic statistics for Atlanta is as follows:

https://www.basketball-reference.com/teams/ATL/2017/gamelog/

And the advanced statistics:

https://www.basketball-reference.com/teams/ATL/2017/gamelog-advanced/

The address for each team is essentially the same, except for the unique three letter abbreviation (ATL in Atlanta's case).

We need to iterate through the basketball-reference home page to grab the 3 letter abbreviation for each team.

In [2]:
url = 'https://www.basketball-reference.com/'

#request access to the web page
res = requests.get(url)

#assign all of the information to a variable
soup = BeautifulSoup(res.content, 'lxml')

#create an empty list to put the teams in
teams = []

#grab the 3 letter abbreviation for each NBA team
for conf in ['E', 'W']:
    table = soup.find('table', {'id': 'confs_standings_'+conf})
    for row in table.find('tbody').find_all('tr'):
        teams.append(row.find('a').text)

#order alphabetically
teams = sorted(teams)

In [3]:
#check to make sure all 30 teams are in there
print(len(teams))
print(teams)

30
['ATL', 'BOS', 'BRK', 'CHI', 'CHO', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS']


In [4]:
#use the abbreviations to create the unique urls for each team
team_urls = []
team_adv_urls = []
for team in teams:
    team_urls.append('https://www.basketball-reference.com/teams/' \
                     + team + '/2017/gamelog/')
    
    team_adv_urls.append('https://www.basketball-reference.com/teams/' \
                         + team + '/2017/gamelog-advanced/')

#check that the lists are populated
print([len(team_urls), len(team_adv_urls)])
display(team_urls[:5])
display(team_adv_urls[:5])

[30, 30]


['https://www.basketball-reference.com/teams/ATL/2017/gamelog/',
 'https://www.basketball-reference.com/teams/BOS/2017/gamelog/',
 'https://www.basketball-reference.com/teams/BRK/2017/gamelog/',
 'https://www.basketball-reference.com/teams/CHI/2017/gamelog/',
 'https://www.basketball-reference.com/teams/CHO/2017/gamelog/']

['https://www.basketball-reference.com/teams/ATL/2017/gamelog-advanced/',
 'https://www.basketball-reference.com/teams/BOS/2017/gamelog-advanced/',
 'https://www.basketball-reference.com/teams/BRK/2017/gamelog-advanced/',
 'https://www.basketball-reference.com/teams/CHI/2017/gamelog-advanced/',
 'https://www.basketball-reference.com/teams/CHO/2017/gamelog-advanced/']

## Start by Getting the Column Names

The first function will get the relevant column names.  The web page to be scraped is the input to this function.
Beautiful Soup is the web scrape module.  I am including an if statement in the function in case a server error occurs during execution.  If there's an error, it will print whether it is a 400 or 500 error.  The former means that there was a mistake on the user's end, the latter means the site is experiencing issues.

There are multiple headers on the table that contains the team statistics.  Some we do not need, so the for loop checks each header and passes on those.  It also transforms the column names into a format that makes it easier to work with in pandas by lower casing the column names, adding an underscore where there are spaces, and changing the % sign to "pct".

In [5]:
def team_column_names(url):
    #access the site with the url
    url = url
    res = requests.get(url)
    
    #if statement to check and make sure the url was entered properly
    if res.status_code == 200:
        soup = BeautifulSoup(res.content, 'lxml')
        
        #accessing the column names from the table
        headers = soup.find_all('table')[0]('thead')[0]('th')
        
        #a list of the headers we do not want 
        to_drop = to_drop = ['Team', 'Opponent', 'Advanced', 
                             'Offensive Four Factors', 'Defensive Four Factors', 'Rk']
        
        #set up an empty list to put the column names in
        columns = []
        
        #skip unneccessary headers, clean column names
        for i in range(len(headers)):
            if len(headers[i].text) == 0:
                pass
            elif headers[i].text in to_drop:
                pass
            elif '%' in headers[i].text:
                columns.append(headers[i].text.lower().replace('%', '_pct'))
            elif '/' in headers[i].text:
                columns.append(headers[i].text.lower().replace('/', '_'))
            else:
                columns.append(headers[i].text.lower())
                
    #if the url generates an error this will let us know what kind            
    else:
        print('There was a ' + res.status_code + ' error')
        
    return columns

## Test the Function

I'll grab the first team in the url lists and run the function to see if it produces the column names I want.  You'll notice that ```'xa0'``` appears as a column name a few times.  Those represent columns in the table where there was data, but no column name.  This will be fixed in one of the functions that follow.

In [7]:
cols = team_column_names(team_urls[0])
adv_cols = team_column_names(team_adv_urls[0])
print(cols)
print(adv_cols)

['g', 'date', '\xa0', 'opp', 'w_l', 'tm', 'opp', 'fg', 'fga', 'fg_pct', '3p', '3pa', '3p_pct', 'ft', 'fta', 'ft_pct', 'orb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', '\xa0', 'fg', 'fga', 'fg_pct', '3p', '3pa', '3p_pct', 'ft', 'fta', 'ft_pct', 'orb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf']
['g', 'date', '\xa0', 'opp', 'w_l', 'tm', 'opp', 'ortg', 'drtg', 'pace', 'ftr', '3par', 'ts_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', '\xa0', 'efg_pct', 'tov_pct', 'orb_pct', 'ft_fga', '\xa0', 'efg_pct', 'tov_pct', 'drb_pct', 'ft_fga']


## The Problem of Duplicate Columns

On the table of basic team stats, the stats for the opponent are listed as well, and they have the same column names.
Instead of having two columns marked ```'fg'```, I will change them to ```'team_fg'``` and ```'opp_fg'```.  In order to do that, I need to add a unique number to the end of the column name so that I can rename them one at a time.  This will make sense when you see the output of the next function.

In [1]:
#import counter to give a unique label to duplicate columns
from collections import Counter

def team_column_rename(columns):
    
    #creates a dictionary with column name as key and times it occurs as the value
    name_dict = Counter(columns)
    
    #this grabs duplicate column names
    keys = [key for key in name_dict if name_dict[key] > 1]
    
    #a counter to add a unique number to the end of duplicate column names
    count = 1
    cols = []
    for name in columns:
        if name in keys:
            cols.append(name + '_' + str(count))
            count += 1
        else:
            cols.append(name)

    return cols

Here we make sure that each duplicate column now has a unique number label.  This way in the next funtion we can rename them to what we want.

In [9]:
cols = team_column_rename(cols)
adv_cols = team_column_rename(adv_cols)
print(cols)
print(adv_cols)

['g', 'date', '\xa0_1', 'opp_2', 'w_l', 'tm', 'opp_3', 'fg_4', 'fga_5', 'fg_pct_6', '3p_7', '3pa_8', '3p_pct_9', 'ft_10', 'fta_11', 'ft_pct_12', 'orb_13', 'trb_14', 'ast_15', 'stl_16', 'blk_17', 'tov_18', 'pf_19', '\xa0_20', 'fg_21', 'fga_22', 'fg_pct_23', '3p_24', '3pa_25', '3p_pct_26', 'ft_27', 'fta_28', 'ft_pct_29', 'orb_30', 'trb_31', 'ast_32', 'stl_33', 'blk_34', 'tov_35', 'pf_36']
['g', 'date', '\xa0_1', 'opp_2', 'w_l', 'tm', 'opp_3', 'ortg', 'drtg', 'pace', 'ftr', '3par', 'ts_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', '\xa0_4', 'efg_pct_5', 'tov_pct_6', 'orb_pct', 'ft_fga_7', '\xa0_8', 'efg_pct_9', 'tov_pct_10', 'drb_pct', 'ft_fga_11']


## Grab the Table Data

Now that each column has a unique name, we can rename them to reflect whether they are team stats or opposing team stats.  Before we can do that, we need a dataframe.  We have column names, now it's time to get the data from the table.

The table has a few lines that we want to skip.  For example, every 20 lines, the table headers appear again.  Fortunately those duplicate header lines have a length of zero.  So, in order to keep the for loop from breaking and returning an error, we check the length of each row to make sure it equals the number of columns in the dataframe.

In [2]:
#pass in the url
def get_team_data(url):
    
    #same code as in the above functions
    url = url
    res = requests.get(url)
    if res.status_code == 200:
        soup = BeautifulSoup(res.content, 'lxml')
        
        #empty list for each data point to go in
        data = []
        
        #the number of rows that will be in the dataframe
        num_rows = len(soup.find_all('table')[0]('tbody')[0]('tr'))
        
        #the number of columns
        num_cols = len(soup.find_all('table')[0]('tbody')[0]('tr')[0]('td'))

        
        #iterates through each row and grabs the appropriate value for each column
        #and throws it in one long list
        for x in range(num_rows):
            for y in range(num_cols):

                if len(soup.find_all('table')[0]('tbody')[0]('tr')[x]('td')) == num_cols:
                    data.append(soup.find_all('table')[0]('tbody')[0]('tr')[x]('td')[y].text)
                
                #if the line is not the correct length, skip it
                else:
                    pass
        
        #breaks the data up into a list of lists each the same length as 
        #the number of columns
        data = [data[x:x+num_cols] for x in range(0, len(data), num_cols)]
    
    return data

There are 82 games in an NBA season, so there should be 82 lists of data, each with a length that matches the number of columns.

In [11]:
team_data = get_team_data(team_urls[0])
adv_team_data = get_team_data(team_adv_urls[0])

print(len(team_data))
print(len(team_data[0]) == (len(cols)))

82
True


## Combine into One Dataframe

Next step is to combine the columns and the data to make a dataframe.  You'll start to notice that the functions going forward will use the previous functions.  That way, the process of gathering all the team data can be automated.

In [12]:
#input the web pages for both basic team stats and advanced team stats 
def team_combine(url_1, url_2):
    
    #apply the three previous functions on the url to get a dataframe
    
    #basic stats
    df_1 = pd.DataFrame(data=get_team_data(url_1),
                        columns=team_column_rename(team_column_names(url_1)))
    
    #advanced stats
    df_2 = pd.DataFrame(data=get_team_data(url_2),
                       columns=team_column_rename(team_column_names(url_2)))
    
    #there are 7 columns that are the exact same in both dfs
    #we leave those columns out of df_2 when we merge
    merge_cols = [x for x in df_2.columns if x not in df_1.columns[:7]]
    
    return pd.merge(df_1, df_2[merge_cols],
                   left_index=True, right_index=True)

Now, with these four functions, we can input the basic and advanced web pages into the ```team_combine``` function and it will output a dataframe.

In [13]:
#this code will keep the df from abbreviating the amount of columns it displays
pd.set_option('display.max_columns', 300)


atlanta = team_combine(team_urls[0], team_adv_urls[0])
atlanta.head()

,g,date,_1,opp_2,w_l,tm,opp_3,fg_4,fga_5,fg_pct_6,3p_7,3pa_8,3p_pct_9,ft_10,fta_11,ft_pct_12,orb_13,trb_14,ast_15,stl_16,blk_17,tov_18,pf_19,_20,fg_21,fga_22,fg_pct_23,3p_24,3pa_25,3p_pct_26,ft_27,fta_28,ft_pct_29,orb_30,trb_31,ast_32,stl_33,blk_34,tov_35,pf_36,ortg,drtg,pace,ftr,3par,ts_pct,trb_pct,ast_pct,stl_pct,blk_pct,_4,efg_pct_5,tov_pct_6,orb_pct,ft_fga_7,_8,efg_pct_9,tov_pct_10,drb_pct,ft_fga_11
0,1,2016-10-27,,WAS,W,114,99,44,88,.500,12,26,.462,14,18,.778,14,52,28,13,7,21,19,,40,94,.426,6,20,.300,13,18,.722,12,40,25,14,4,19,20,110.2,95.7,103.4,.205,.295,.594,56.5,63.6,12.6,9.5,,.568,18.0,33.3,.159,,.457,15.7,76.0,.138
1,2,2016-10-29,@,PHI,W,104,72,42,85,.494,5,20,.250,15,23,.652,6,47,31,11,7,12,14,,28,80,.350,5,18,.278,11,15,.733,4,39,16,9,5,20,20,103.7,71.8,100.3,.271,.235,.547,54.7,73.8,11.0,11.3,,.524,11.2,14.6,.176,,.381,18.8,91.1,.138
2,3,2016-10-31,,SAC,W,106,95,33,79,.418,11,30,.367,29,47,.617,17,43,24,11,6,13,20,,36,75,.480,10,23,.435,13,19,.684,6,38,26,10,6,19,34,113.1,101.3,93.7,.595,.380,.532,53.1,72.7,11.7,11.5,,.487,11.5,34.7,.367,,.547,18.6,81.3,.173
3,4,2016-11-02,,LAL,L,116,123,41,85,.482,11,32,.344,23,29,.793,10,37,30,10,1,18,19,,48,90,.533,9,22,.409,18,20,.900,11,42,24,11,9,15,25,114.2,121.1,101.6,.341,.376,.593,46.8,73.2,9.8,1.5,,.547,15.5,24.4,.271,,.583,13.2,71.1,.200
4,5,2016-11-04,@,WAS,L,92,95,32,83,.386,5,25,.200,23,36,.639,10,47,22,12,3,18,24,,31,80,.388,5,20,.250,28,35,.800,11,46,13,7,3,19,23,90.1,93.0,102.1,.434,.301,.465,50.5,68.8,11.7,5.0,,.416,15.4,22.2,.277,,.419,16.6,77.1,.350


## Rename the Columns

Now that we have a dataframe, we can give it appropriate column names.  You may notice above that there are three columns with no data in them.  This function will drop those columns.  Again, this function incorporates all of the previous functions when we input the basic and advanced statistics pages.

In [14]:
def clean_team_columns(url_1, url_2):
    
    #applies all previous functions
    df = team_combine(url_1, url_2)
    
    df.rename(columns={'g': 'opp_game', df.columns[2]: 'opp_home', 'opp_2': 'team',
                           'w_l': 'opp_win', 'tm': 'opp_score', 'opp_3': 'score',
                           'fg_4': 'opp_fg', 'fga_5': 'opp_fga', 'fg_pct_6': 'opp_fg_pct',
                           '3p_7': 'opp_3p', '3pa_8': 'opp_3pa', '3p_pct_9': 'opp_3p_pct',
                           'ft_10': 'opp_ft', 'fta_11': 'opp_fta', 'ft_pct_12': 'opp_ft_pct',
                           'orb_13': 'opp_orb', 'trb_14': 'opp_trb', 'ast_15': 'opp_ast',
                           'stl_16': 'opp_stl', 'blk_17': 'opp_blk', 'tov_18': 'opp_tov',
                           'pf_19': 'opp_pf', 'fg_21': 'team_fg', 'fga_22': 'team_fga',
                           'fg_pct_23': 'team_fg_pct', '3p_24': 'team_3p', '3pa_25': 'team_3pa',
                           '3p_pct_26': 'team_3p_pct', 'ft_27': 'team_ft', 'fta_28': 'team_fta',
                           'ft_pct_29': 'team_ft_pct', 'orb_30': 'team_orb', 'trb_31': 'team_trb',
                           'ast_32': 'team_ast', 'stl_33': 'team_stl', 'blk_34': 'team_blk',
                           'tov_35': 'team_tov', 'pf_36': 'team_pf', 'ortg': 'opp_ortg',
                           'drtg': 'opp_drtg', 'pace': 'opp_pace', 'ftr': 'opp_ftr',
                           '3par': 'opp_3par', 'ts_pct': 'opp_ts_pct', 'ts_pct': 'opp_ts_pct',
                           'trb_pct': 'opp_trb_pct', 'ast_pct': 'opp_ast_pct', 'stl_pct': 'opp_stl_pct',
                           'blk_pct': 'opp_blk_pct', 'efg_pct_5': 'opp_off_efg_pct', 
                           'tov_pct_6': 'opp_off_tov_pct', 'orb_pct': 'opp_orb_pct',
                           'ft_fga_7': 'opp_off_ft_fga', 'efg_pct_9': 'opp_def_efg_pct',
                           'tov_pct_10': 'opp_def_tov_pct', 'drb_pct': 'opp_drb_pct', 
                            'ft_fga_11': 'opp_def_ft_fga'}, inplace=True)

    df.drop(columns=[df.columns[23], df.columns[50],
                          df.columns[55]], axis=1, inplace=True)
    
    return df

Check to see that columns have the correct names.

In [15]:
atlanta = clean_team_columns(team_urls[0], team_adv_urls[0])
atlanta.set_index('date', inplace=True)
atlanta.head()

,opp_game,opp_home,team,opp_win,opp_score,score,opp_fg,opp_fga,opp_fg_pct,opp_3p,opp_3pa,opp_3p_pct,opp_ft,opp_fta,opp_ft_pct,opp_orb,opp_trb,opp_ast,opp_stl,opp_blk,opp_tov,opp_pf,team_fg,team_fga,team_fg_pct,team_3p,team_3pa,team_3p_pct,team_ft,team_fta,team_ft_pct,team_orb,team_trb,team_ast,team_stl,team_blk,team_tov,team_pf,opp_ortg,opp_drtg,opp_pace,opp_ftr,opp_3par,opp_ts_pct,opp_trb_pct,opp_ast_pct,opp_stl_pct,opp_blk_pct,opp_off_efg_pct,opp_off_tov_pct,opp_orb_pct,opp_off_ft_fga,opp_def_efg_pct,opp_def_tov_pct,opp_drb_pct,opp_def_ft_fga
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-10-27,1,,WAS,W,114,99,44,88,.500,12,26,.462,14,18,.778,14,52,28,13,7,21,19,40,94,.426,6,20,.300,13,18,.722,12,40,25,14,4,19,20,110.2,95.7,103.4,.205,.295,.594,56.5,63.6,12.6,9.5,.568,18.0,33.3,.159,.457,15.7,76.0,.138
2016-10-29,2,@,PHI,W,104,72,42,85,.494,5,20,.250,15,23,.652,6,47,31,11,7,12,14,28,80,.350,5,18,.278,11,15,.733,4,39,16,9,5,20,20,103.7,71.8,100.3,.271,.235,.547,54.7,73.8,11.0,11.3,.524,11.2,14.6,.176,.381,18.8,91.1,.138
2016-10-31,3,,SAC,W,106,95,33,79,.418,11,30,.367,29,47,.617,17,43,24,11,6,13,20,36,75,.480,10,23,.435,13,19,.684,6,38,26,10,6,19,34,113.1,101.3,93.7,.595,.380,.532,53.1,72.7,11.7,11.5,.487,11.5,34.7,.367,.547,18.6,81.3,.173
2016-11-02,4,,LAL,L,116,123,41,85,.482,11,32,.344,23,29,.793,10,37,30,10,1,18,19,48,90,.533,9,22,.409,18,20,.900,11,42,24,11,9,15,25,114.2,121.1,101.6,.341,.376,.593,46.8,73.2,9.8,1.5,.547,15.5,24.4,.271,.583,13.2,71.1,.200
2016-11-04,5,@,WAS,L,92,95,32,83,.386,5,25,.200,23,36,.639,10,47,22,12,3,18,24,31,80,.388,5,20,.250,28,35,.800,11,46,13,7,3,19,23,90.1,93.0,102.1,.434,.301,.465,50.5,68.8,11.7,5.0,.416,15.4,22.2,.277,.419,16.6,77.1,.350


## Transform the Numeric Data Types and Add Average Columns

Ultimately the model features will be either season averages of each stat up to the date of the current game, or the average of each stat in the previous five games.  This function performs the ```.rolling()``` function on all of the numeric columns, and it shifts the rows by one.  There is also code to get columns that represent the season to date averages of all the numeric columns.

Some of the column values like whether or not the team played at home, and whether or not they won need to be represented as a 1 or 0.  Finally, we make the date the index so that we can eventually merge a player's data with the from their opponent on that date.

In [51]:
def add_team_avgs(url_1, url_2):
    
    #apply all previous functions
    df = clean_team_columns(url_1, url_2)

    #change these columns to 1s or 0s
    df['opp_home'] = df['opp_home'].map(lambda x: 1 if '@' in x else 0)
    df['opp_win'] = df['opp_win'].map(lambda x: 1 if 'W' in x else 0)
    
    #make the date the index
    df.set_index('date', inplace=True)

    #list of all the column names except for the string column "team"
    to_average = [x for x in df.columns if x != 'team']

    #change the data types from object to int or float
    for x in to_average:
        df[x] = pd.to_numeric(df[x])

    #create new columns for season to date average and five game average
    for stat in to_average:
        avgs = []
        for i in range(len(df[stat])):
            avgs.append((sum(df[stat][:(i + 1)])) / len(df[stat][:(i + 1)]))

        #make sure to shift
        df['avg_' + stat] = avgs
        df['avg_' + stat] = df['avg_' + stat].round(2).shift(1)

        df['roll5_' + stat] = df[stat].rolling(5).mean().round(2).shift(1)
        
    return df

Run the function and check the data types

In [17]:
atlanta = add_team_avgs(team_urls[0], team_adv_urls[0])
atlanta.dtypes.value_counts()

float64    134
int64       31
object       1
dtype: int64

## Apply to Every Team

Now that we know each function is working properly, we can loop through the rest of the teams, and create one big dataframe.  This will take around a half hour.  We add the ```time``` module to let us know how long the cell took to execute.  I am also adding a print statement to show the progress.

In [18]:
start = time.time()

#create a list for each team df, put atlanta in there since we have
#already gathered the data for them
df_list = [atlanta]

#iterate through the list of team urls, starting with the second team
for i in range(1, len(team_urls)):
    #add each team to df_list
    df_list.append(add_team_avgs(team_urls[i], team_adv_urls[i]))
    
    #print out the progress of the for-loop
    print('Team ' + str(i+1) + ' of 30 has been added ' + '(' + teams[i] + ')')
    
end = time.time()
print("Loaded data in {} minutes".format((end - start) / 60.))

Team 2 of 30 has been added (BOS)
Team 3 of 30 has been added (BRK)
Team 4 of 30 has been added (CHI)
Team 5 of 30 has been added (CHO)
Team 6 of 30 has been added (CLE)
Team 7 of 30 has been added (DAL)
Team 8 of 30 has been added (DEN)
Team 9 of 30 has been added (DET)
Team 10 of 30 has been added (GSW)
Team 11 of 30 has been added (HOU)
Team 12 of 30 has been added (IND)
Team 13 of 30 has been added (LAC)
Team 14 of 30 has been added (LAL)
Team 15 of 30 has been added (MEM)
Team 16 of 30 has been added (MIA)
Team 17 of 30 has been added (MIL)
Team 18 of 30 has been added (MIN)
Team 19 of 30 has been added (NOP)
Team 20 of 30 has been added (NYK)
Team 21 of 30 has been added (OKC)
Team 22 of 30 has been added (ORL)
Team 23 of 30 has been added (PHI)
Team 24 of 30 has been added (PHO)
Team 25 of 30 has been added (POR)
Team 26 of 30 has been added (SAC)
Team 27 of 30 has been added (SAS)
Team 28 of 30 has been added (TOR)
Team 29 of 30 has been added (UTA)
Team 30 of 30 has been added

In [19]:
teams_df = pd.concat(df_list)

There are 30 teams in the NBA, each plays 82 games.  There should be 2,460 rows in this dataframe.

In [20]:
teams_df.shape

(2460, 166)

Save the completed team data to a .csv file.

In [21]:
teams_df.to_csv('./Data/teams/team17.csv')

## Now For the Player Data

Just like before, we will create a series of functions to scrape, clean, and transform individual player data.  There are a lot of similarities to the team functions, but enough differences that we need to write seperate ones for the players.

## Get Unique Url For Each Player

This involves slightly more work than the team abbreviations did.  Each player is scraped from the page:

https://www.basketball-reference.com/leagues/NBA_2017_per_game.html

Here's what the basic per game stats url looks like for the first player on the list, Alex Abrines:

https://www.basketball-reference.com/players/a/abrinal01/gamelog/2017

And the advanced per game stats:

https://www.basketball-reference.com/players/a/abrinal01/gamelog-advanced/2017/

To following code will iterate through the list of players to get their name and unique id("a/abrinal01" in the case of Alex Abrines).

A player appears more than once in the table if they played for more than one team during the season, so we will have to account for that.

In [6]:
stats = 'https://www.basketball-reference.com/leagues/NBA_2017_per_game.html'
res = requests.get(stats)
soup = BeautifulSoup(res.content, 'lxml')

#this module helps us make sure there's no duplicate players in the list
from collections import OrderedDict


header = len(soup.find_all('table')[0]('thead')[0]('tr')[0]('th'))
num_rows = len(soup.find_all('table')[0]('tbody')[0]('tr'))

players = []

#the if statement makes sure code won't break when it gets to a line without a player in it
for x in range(num_rows):
    if len(soup.find_all('table')[0]('tbody')[0]('tr')[x]('td')) == (header - 1):
        players.append(soup.find_all('table')[0]('tbody')[0]('tr')[x]('td')[0].text)
    else:
        pass
    

#eliminate duplicates    
players = list(OrderedDict.fromkeys(players))
players[:5]

['Alex Abrines',
 'Quincy Acy',
 'Steven Adams',
 'Arron Afflalo',
 'Alexis Ajinca']

In [7]:
#now get unique player ids

header = len(soup.find_all('table')[0]('thead')[0]('tr')[0]('th'))
num_rows = len(soup.find_all('table')[0]('tbody')[0]('tr'))

player_ids = []

for x in range(num_rows):
    if len(soup.find_all('table')[0]('tbody')[0]('tr')[x]('td')) == (header - 1):
            player_ids.append(soup.find_all('table')[0]('tbody')[0]('tr')[x]('td')[0]('a')[0]['href'])
    else:
        pass

player_ids = [x.replace('.html', '') for x in player_ids]
player_ids = list(OrderedDict.fromkeys(player_ids))
player_ids[:5]

['/players/a/abrinal01',
 '/players/a/acyqu01',
 '/players/a/adamsst01',
 '/players/a/afflaar01',
 '/players/a/ajincal01']

Now create the unique url for each player's basic and advanced statistics page.

In [8]:
player_urls = []
player_adv_urls = []

for player in player_ids:
    player_urls.append('https://www.basketball-reference.com' + player \
                       + '/gamelog/2017')
    
for player in player_ids:
    player_adv_urls.append('https://www.basketball-reference.com' + player \
                           + '/gamelog-advanced/2017')
    
display(player_urls[:5])
display(player_adv_urls[:5])

['https://www.basketball-reference.com/players/a/abrinal01/gamelog/2017',
 'https://www.basketball-reference.com/players/a/acyqu01/gamelog/2017',
 'https://www.basketball-reference.com/players/a/adamsst01/gamelog/2017',
 'https://www.basketball-reference.com/players/a/afflaar01/gamelog/2017',
 'https://www.basketball-reference.com/players/a/ajincal01/gamelog/2017']

['https://www.basketball-reference.com/players/a/abrinal01/gamelog-advanced/2017',
 'https://www.basketball-reference.com/players/a/acyqu01/gamelog-advanced/2017',
 'https://www.basketball-reference.com/players/a/adamsst01/gamelog-advanced/2017',
 'https://www.basketball-reference.com/players/a/afflaar01/gamelog-advanced/2017',
 'https://www.basketball-reference.com/players/a/ajincal01/gamelog-advanced/2017']

There were 486 players who played in the 2016-2017 season.  Make sure that's how many urls we have.

In [27]:
print(len(player_urls))
print(len(player_adv_urls))

486
486


## Get The Player Column Names

Unlike with the team web pages where the basic and advanced stats were located on the same table index, the basic player stats are on table index 7 and the advanced stats are on table index zero.  Due to this, the column and data functions take two inputs.

In [9]:
""" 

There is no 'to_drop' list in this function like there was 
in the team function.  The only header we are not interested in is 'Rk' 

"""

def player_column_names(url, table_index):
    
    url = url
    res = requests.get(url)
    if res.status_code == 200:
        soup = BeautifulSoup(res.content, 'lxml')
        headers = soup.find_all('table')[table_index]('thead')[0]('th')
        
        columns = []
        for i in range(len(headers)):
            if len(headers[i].text) == 0:
                pass
            elif headers[i].text == 'Rk':
                pass
            elif '%' in headers[i].text:
                columns.append(headers[i].text.lower().replace('%', '_pct'))
            elif '/' in headers[i].text:
                columns.append(headers[i].text.lower().replace('/', '_'))
            else:
                columns.append(headers[i].text.lower())
    else:
        print('There was a ' + res.status_code + ' error')
        
    return columns

In [10]:
#input url and corresponding table index
abrines_cols = player_column_names(player_urls[0], 7)
abrines_adv_cols = player_column_names(player_adv_urls[0], 0)
print(abrines_cols)
print(abrines_adv_cols)

['g', 'date', 'age', 'tm', '\xa0', 'opp', '\xa0', 'gs', 'mp', 'fg', 'fga', 'fg_pct', '3p', '3pa', '3p_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'gmsc', '+_-']
['g', 'date', 'age', 'tm', '\xa0', 'opp', '\xa0', 'gs', 'mp', 'ts_pct', 'efg_pct', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ortg', 'drtg', 'gmsc']


In [30]:
"""

Fortunately, there are not nearly as many duplicate columns
with the players as there were with the teams.  The code
here is essentially the same as the team version, but to
make things less confusing we'll make a new function for the
player data

"""
def player_column_rename(columns):
    
    name_dict = Counter(columns)
    keys = [key for key in name_dict if name_dict[key] > 1]
    count = 1
    cols = []
    for name in columns:
        if name in keys:
            cols.append(name + '_' + str(count))
            count += 1
        else:
            cols.append(name)

    return cols

In [31]:
abrines_cols = player_column_rename(abrines_cols)
abrines_adv_cols = player_column_rename(abrines_adv_cols)
print(abrines_cols)
print(abrines_adv_cols)

['g', 'date', 'age', 'tm', '\xa0_1', 'opp', '\xa0_2', 'gs', 'mp', 'fg', 'fga', 'fg_pct', '3p', '3pa', '3p_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'gmsc', '+_-']
['g', 'date', 'age', 'tm', '\xa0_1', 'opp', '\xa0_2', 'gs', 'mp', 'ts_pct', 'efg_pct', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ortg', 'drtg', 'gmsc']


## Get Player Data

In [32]:
def player_data(url, table_index):
    
    url = url
    res = requests.get(url)
    if res.status_code == 200:
        soup = BeautifulSoup(res.content, 'lxml')
        data = []
        num_rows = len(soup.find_all('table')[table_index]('tbody')[0]('tr'))
        num_cols = len(player_column_names(url, table_index))

        for x in range(num_rows):
            for y in range(num_cols):

                if len(soup.find_all('table')[table_index]('tbody')[0]('tr')[x]('td')) == num_cols:
                    data.append(soup.find_all('table')[table_index]('tbody')[0]('tr')[x]('td')[y].text)
                else:
                    pass
        data = [data[x:x+num_cols] for x in range(0, len(data), num_cols)]
    
    return data
    

Alex Abrines played in 68 games in 2016-2 017, so that's how long the list of row lists should be.  There are 29 basic stat columns, and 22 advance stat columns.

In [33]:
abrines_data = player_data(player_urls[0], 7)
abrines_adv_data = player_data(player_adv_urls[0], 0)

#check that all the data is in there
print(len(abrines_data), len(abrines_data[0]))
print(len(abrines_adv_data), len(abrines_adv_cols))

68 29
68 22


## Combine the Basic and Advanced Player Stats

This is slightly different than the team version, the first nine columns for both are the same (it was 7 with the team data) but also, both dfs have a columns called 'gmsc', but it's in a different spot in each.

In [52]:
def player_combine(url_1, url_2):
    
    df_1 = pd.DataFrame(data=player_data(url_1, 7),
                        columns=player_column_rename(player_column_names(url_1, 7)))
    
    df_2 = pd.DataFrame(data=player_data(url_2, 0),
                       columns=player_column_rename(player_column_names(url_2, 0)))
    
    
    #do not include duplicate columns in the merge
    duplicates = list(df_1.columns[:9])
    
    #adding the gsmc column to the duplicates list
    duplicates.append(df_1.columns[-2])
    
    merge_cols = [x for x in df_2.columns if x not in duplicates]
    
    return pd.merge(df_1, df_2[merge_cols],
                   left_index=True, right_index=True)

Make sure we get 1 dataframe from the function.

In [35]:
abrines = player_combine(player_urls[0], player_adv_urls[0])
print(abrines.shape)
display(abrines.head())

(68, 41)


,g,date,age,tm,_1,opp,_2,gs,mp,fg,fga,fg_pct,3p,3pa,3p_pct,ft,fta,ft_pct,orb,drb,trb,ast,stl,blk,tov,pf,pts,gmsc,+_-,ts_pct,efg_pct,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,ortg,drtg
0,1,2016-10-26,23-086,OKC,@,PHI,W (+6),0,13:24,1,2,.500,1,1,1.000,0,0,,0,1,1,0,0,0,2,3,3,-0.9,+5,.750,.750,0.0,8.0,3.7,0.0,0.0,0.0,50.0,12.4,64,101
1,2,2016-10-30,23-090,OKC,,LAL,W (+17),0,1:49,1,1,1.000,0,0,,3,3,1.000,0,0,0,0,1,0,0,0,5,5.7,0,1.078,1.000,0.0,0.0,0.0,0.0,25.0,0.0,0.0,50.5,235,49
2,3,2016-11-02,23-093,OKC,@,LAC,W (+2),0,7:54,2,2,1.000,2,2,1.000,0,0,,0,0,0,0,0,0,2,0,6,3.4,+4,1.500,1.500,0.0,0.0,0.0,0.0,0.0,0.0,50.0,21.4,101,91
3,4,2016-11-03,23-094,OKC,@,GSW,L (-26),0,19:20,1,6,.167,1,6,.167,2,2,1.000,1,2,3,1,0,0,0,0,5,3.2,-13,.363,.250,5.6,11.5,8.6,7.6,0.0,0.0,0.0,14.8,100,132
4,5,2016-11-05,23-096,OKC,,MIN,W (+20),0,17:16,0,1,.000,0,0,,1,1,1.000,0,2,2,2,1,0,0,1,1,2.9,-4,.347,.000,0.0,11.8,6.8,12.4,3.0,0.0,0.0,3.7,132,99


## Clean the Player Data

There are not nearly as many columns to rename as there were in the team data.  We are performing the data type conversion in this function instead of the next one like we did with the team data, because in the final player function there's more columns to add and mathematical operations to perform.

In [53]:
def player_clean_columns(url_1, url_2):
    
    df = player_combine(url_1, url_2)
    
    df.rename(columns={'g': 'gp', 'tm': 'team', df.columns[4]: 'home',
                       df.columns[6]: 'win', 'gs': 'started',
                       '+_-': 'plus_minus'}, inplace=True)

    df.home = df.home.map(lambda x: 1 if x in ['@'] else 0)

    df.win = df.win.map(lambda x: 1 if 'W' in x else 0)
    
    #this is to display player age in just years, not years and days
    df.age = df.age.map(lambda x: x[:2])

    #this rounds minutes played to the minutes and leaves off seconds
    df.mp = df.mp.map(lambda x: int(float(x.replace(':', '.'))) if ':' in x else x)
    
    #convert dtypes
    non_numeric = ['date', 'team', 'opp']
    convert = [x for x in df.columns if x not in non_numeric]

    for x in convert:
        df[x] = pd.to_numeric(df[x])

    #set index to date so we can eventually add opponent data
    df.set_index('date', inplace=True)

    #the dtype conversion puts null values if a player didn't take any three point shots
    #or free throws, so we fill the null values with zeros
    df.fillna(0.00, inplace=True)
    
    return df

Make sure we have the data types we want

In [39]:
abrines = player_clean_columns(player_urls[0], player_adv_urls[0])
print(abrines.dtypes.value_counts())
display(abrines.head())

int64      24
float64    14
object      2
dtype: int64


,gp,age,team,home,opp,win,started,mp,fg,fga,fg_pct,3p,3pa,3p_pct,ft,fta,ft_pct,orb,drb,trb,ast,stl,blk,tov,pf,pts,gmsc,plus_minus,ts_pct,efg_pct,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,ortg,drtg
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-10-26,1,23,OKC,1,PHI,1,0,13,1,2,0.500,1,1,1.000,0,0,0.0,0,1,1,0,0,0,2,3,3,-0.9,5,0.750,0.75,0.0,8.0,3.7,0.0,0.0,0.0,50.0,12.4,64,101
2016-10-30,2,23,OKC,0,LAL,1,0,1,1,1,1.000,0,0,0.000,3,3,1.0,0,0,0,0,1,0,0,0,5,5.7,0,1.078,1.00,0.0,0.0,0.0,0.0,25.0,0.0,0.0,50.5,235,49
2016-11-02,3,23,OKC,1,LAC,1,0,7,2,2,1.000,2,2,1.000,0,0,0.0,0,0,0,0,0,0,2,0,6,3.4,4,1.500,1.50,0.0,0.0,0.0,0.0,0.0,0.0,50.0,21.4,101,91
2016-11-03,4,23,OKC,1,GSW,0,0,19,1,6,0.167,1,6,0.167,2,2,1.0,1,2,3,1,0,0,0,0,5,3.2,-13,0.363,0.25,5.6,11.5,8.6,7.6,0.0,0.0,0.0,14.8,100,132
2016-11-05,5,23,OKC,0,MIN,1,0,17,0,1,0.000,0,0,0.000,1,1,1.0,0,2,2,2,1,0,0,1,1,2.9,-4,0.347,0.00,0.0,11.8,6.8,12.4,3.0,0.0,0.0,3.7,132,99


## Add Columns for Double-Double, Triple_Double, and Fantasy Points

These are very important aspects to a daily fantasy score and they were not included in the tables that we scraped out data from, so we must create them.  How these scores are calculated is explained in the technical report.  Along with the new columns, we will create season to date average and 5-game average columns as well for every numerical column.

In [40]:
def add_player_columns(url_1, url_2):
    
    df = player_clean_columns(url_1, url_2)
    
    #these are columns that do not need to be averaged
    leave_out = ['gp', 'age', 'team', 'home',
                 'opp', 'win', 'started']

    #calculating the double-double, triple-double, and fantasy score(target variable) columns
    stats = ['pts', 'trb', 'ast', 'blk', 'stl']
    df['trip_dub'] = (df[stats] >= 10).sum(1) >= 3
    df['dub_dub'] = (df[stats] >= 10).sum(1) >= 2
    df['trip_dub'] = df['trip_dub'].map(lambda x: 1 if x == True else 0)
    df['dub_dub'] = df['dub_dub'].map(lambda x: 1 if x == True else 0)
    
    df['fantasy_points'] = (df.pts) \
                            + (df['3p'] * .5) \
                            + (df.trb * 1.25) \
                            + (df.ast * 1.5) \
                            + (df.stl * 2) \
                            + (df.blk * 2) \
                            - (df.tov * .5) \
                            + (df.dub_dub * 1.5) \
                            + (df.trip_dub * 3)
    
    #get season to date and five game averages
    to_average = [x for x in df.columns if x not in leave_out]
    
    for stat in to_average:
        avgs = []
        for i in range(len(df[stat])):
            avgs.append((sum(df[stat][:(i + 1)])) / len(df[stat][:(i + 1)]))

        df['avg_' + stat] = avgs
        df['avg_' + stat] = df['avg_' + stat].round(2).shift(1)

        df['roll5_' + stat] = df[stat].rolling(5).mean().round(2).shift(1)
        
    return df

In [41]:
abrines = add_player_columns(player_urls[0], player_adv_urls[0])
print(abrines.shape)
display(abrines.head())

(68, 115)


,gp,age,team,home,opp,win,started,mp,fg,fga,fg_pct,3p,3pa,3p_pct,ft,fta,ft_pct,orb,drb,trb,ast,stl,blk,tov,pf,pts,gmsc,plus_minus,ts_pct,efg_pct,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,ortg,drtg,trip_dub,dub_dub,fantasy_points,avg_mp,roll5_mp,avg_fg,roll5_fg,avg_fga,roll5_fga,avg_fg_pct,roll5_fg_pct,avg_3p,roll5_3p,avg_3pa,roll5_3pa,avg_3p_pct,roll5_3p_pct,avg_ft,roll5_ft,avg_fta,roll5_fta,avg_ft_pct,roll5_ft_pct,avg_orb,roll5_orb,avg_drb,roll5_drb,avg_trb,roll5_trb,avg_ast,roll5_ast,avg_stl,roll5_stl,avg_blk,roll5_blk,avg_tov,roll5_tov,avg_pf,roll5_pf,avg_pts,roll5_pts,avg_gmsc,roll5_gmsc,avg_plus_minus,roll5_plus_minus,avg_ts_pct,roll5_ts_pct,avg_efg_pct,roll5_efg_pct,avg_orb_pct,roll5_orb_pct,avg_drb_pct,roll5_drb_pct,avg_trb_pct,roll5_trb_pct,avg_ast_pct,roll5_ast_pct,avg_stl_pct,roll5_stl_pct,avg_blk_pct,roll5_blk_pct,avg_tov_pct,roll5_tov_pct,avg_usg_pct,roll5_usg_pct,avg_ortg,roll5_ortg,avg_drtg,roll5_drtg,avg_trip_dub,roll5_trip_dub,avg_dub_dub,roll5_dub_dub,avg_fantasy_points,roll5_fantasy_points
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-10-26,1,23,OKC,1,PHI,1,0,13,1,2,0.500,1,1,1.000,0,0,0.0,0,1,1,0,0,0,2,3,3,-0.9,5,0.750,0.75,0.0,8.0,3.7,0.0,0.0,0.0,50.0,12.4,64,101,0,0,3.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-10-30,2,23,OKC,0,LAL,1,0,1,1,1,1.000,0,0,0.000,3,3,1.0,0,0,0,0,1,0,0,0,5,5.7,0,1.078,1.00,0.0,0.0,0.0,0.0,25.0,0.0,0.0,50.5,235,49,0,0,7.00,13.0,NaN,1.00,NaN,2.00,NaN,0.50,NaN,1.0,NaN,1.00,NaN,1.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,1.00,NaN,1.00,NaN,0.00,NaN,0.00,NaN,0.0,NaN,2.00,NaN,3.00,NaN,3.00,NaN,-0.90,NaN,5.0,NaN,0.75,NaN,0.75,NaN,0.0,NaN,8.00,NaN,3.70,NaN,0.0,NaN,0.00,NaN,0.0,NaN,50.00,NaN,12.40,NaN,64.00,NaN,101.00,NaN,0.0,NaN,0.0,NaN,3.75,NaN
2016-11-02,3,23,OKC,1,LAC,1,0,7,2,2,1.000,2,2,1.000,0,0,0.0,0,0,0,0,0,0,2,0,6,3.4,4,1.500,1.50,0.0,0.0,0.0,0.0,0.0,0.0,50.0,21.4,101,91,0,0,6.00,7.0,NaN,1.00,NaN,1.50,NaN,0.75,NaN,0.5,NaN,0.50,NaN,0.50,NaN,1.50,NaN,1.50,NaN,0.50,NaN,0.00,NaN,0.50,NaN,0.50,NaN,0.00,NaN,0.50,NaN,0.0,NaN,1.00,NaN,1.50,NaN,4.00,NaN,2.40,NaN,2.5,NaN,0.91,NaN,0.88,NaN,0.0,NaN,4.00,NaN,1.85,NaN,0.0,NaN,12.50,NaN,0.0,NaN,25.00,NaN,31.45,NaN,149.50,NaN,75.00,NaN,0.0,NaN,0.0,NaN,5.38,NaN
2016-11-03,4,23,OKC,1,GSW,0,0,19,1,6,0.167,1,6,0.167,2,2,1.0,1,2,3,1,0,0,0,0,5,3.2,-13,0.363,0.25,5.6,11.5,8.6,7.6,0.0,0.0,0.0,14.8,100,132,0,0,10.75,7.0,NaN,1.33,NaN,1.67,NaN,0.83,NaN,1.0,NaN,1.00,NaN,0.67,NaN,1.00,NaN,1.00,NaN,0.33,NaN,0.00,NaN,0.33,NaN,0.33,NaN,0.00,NaN,0.33,NaN,0.0,NaN,1.33,NaN,1.00,NaN,4.67,NaN,2.73,NaN,3.0,NaN,1.11,NaN,1.08,NaN,0.0,NaN,2.67,NaN,1.23,NaN,0.0,NaN,8.33,NaN,0.0,NaN,33.33,NaN,28.10,NaN,133.33,NaN,80.33,NaN,0.0,NaN,0.0,NaN,5.58,NaN
2016-11-05,5,23,OKC,0,MIN,1,0,17,0,1,0.000,0,0,0.000,1,1,1.0,0,2,2,2,1,0,0,1,1,2.9,-4,0.347,0.00,0.0,11.8,6.8,12.4,3.0,0.0,0.0,3.7,132,99,0,0,8.50,10.0,NaN,1.25,NaN,2.75,NaN,0.67,NaN,1.0,NaN,2.25,NaN,0.54,NaN,1.25,NaN,1.25,NaN,0.50,NaN,0.25,NaN,0.75,NaN,1.00,NaN,0.25,NaN,0.25,NaN,0.0,NaN,1.00,NaN,0.75,NaN,4.75,NaN,2.85,NaN,-1.0,NaN,0.92,NaN,0.88,NaN,1.4,NaN,4.88,NaN,3.08,NaN,1.9,NaN,6.25,NaN,0.0,NaN,25.00,NaN,24.78,NaN,125.00,NaN,93.25,NaN,0.0,NaN,0.0,NaN,6.88,NaN


## Add Stats From Each Oppenent to the DataFrame

Since both the player data and the team data is indexed by date, we can merge the two using the date index.  We will also add the player's name to the dataframe.  The below function save each player's final dataframe to a csv file.

In [42]:
def merge_dfs(df_1, df_2, player):
    result = pd.merge(df_1, df_2, on=['date', 'team'])
    new_col = []
    for i in range(len(result)):
        new_col.append(player)

    result.insert(loc=0, column='player', value=new_col)
    
    return result

## Lets Run This on the First Three Players to Make Sure We Get the Correct Result

In [43]:
#track how long this takes to run
start = time.time()

#empty for each completed df
merged_list = []

#iterate through each player
for i in range(len(players[:3])):
    #puts player name in a good format for saving to file
    file_name = players[i].replace(' ', '_').lower()
    merged_list.append(merge_dfs(
                                  add_player_columns(player_urls[i],
                                                     player_adv_urls[i]),
                                                     teams_df,
                                                     players[i])
                      )
    print('Player ' + str(i+1) + ' of ' + str(len(players)) \
          + ' (' + players[i] + ')' + ' has been added')
    merged_list[i].to_csv('./data/'+ str(i) + '_' + file_name + '.csv')

end = time.time()
print("Loaded data in {} minutes".format((end - start) / 60.))

Player 1 of 486 (Alex Abrines) has been added
Player 2 of 486 (Quincy Acy) has been added
Player 3 of 486 (Steven Adams) has been added
Loaded data in 1.7973037004470824 minutes


Adding the name column to the player data makes 116 columns.  The team data has 166.  We merge on the index as well as the common "team" column to get the data for the players opponent on that date.  The final dataframe should have 281 columns (116 plus, 166, minus the one column they have in common.) Alex Abrines played 68 games in the 2017-2018 season, Quincy Acy played 38, and Steven Adams played 80.  So, our dataframe so far should be 186 x 281.

In [44]:
complete_df = pd.concat(merged_list)
complete_df.shape

(186, 281)

## Get the Remaining 483 players' data

This is a time consuming process.  Setting up an EC2 instance on AWS might be worth it, so that this hours-long process does not take up space and time on your local machine.  Letting it run overnight is also an option.

In [45]:
#dont include the empty merged list in this cell so we don't erase
#the three players that are already in there
start = time.time()

for i in range(3, len(players)):
    
    file_name = players[i].replace(' ', '_').lower()
    merged_list.append(merge_dfs(
                                  add_player_columns(player_urls[i],
                                                     player_adv_urls[i]),
                                                     teams_df,
                                                     players[i])
                      )
    print('Player ' + str(i+1) + ' of ' + str(len(players)) \
          + ' (' + players[i] + ')' + ' has been added')
    merged_list[i].to_csv('./data/'+ str(i) + '_' + file_name + '.csv')

end = time.time()
print("Loaded data in {} minutes".format((end - start) / 60.))

Player 4 of 486 (Arron Afflalo) has been added
Player 5 of 486 (Alexis Ajinca) has been added
Player 6 of 486 (Cole Aldrich) has been added
Player 7 of 486 (LaMarcus Aldridge) has been added
Player 8 of 486 (Lavoy Allen) has been added
Player 9 of 486 (Tony Allen) has been added
Player 10 of 486 (Al-Farouq Aminu) has been added
Player 11 of 486 (Chris Andersen) has been added
Player 12 of 486 (Alan Anderson) has been added
Player 13 of 486 (Justin Anderson) has been added
Player 14 of 486 (Kyle Anderson) has been added
Player 15 of 486 (Ryan Anderson) has been added
Player 16 of 486 (Giannis Antetokounmpo) has been added
Player 17 of 486 (Carmelo Anthony) has been added
Player 18 of 486 (Joel Anthony) has been added
Player 19 of 486 (Trevor Ariza) has been added
Player 20 of 486 (Darrell Arthur) has been added
Player 21 of 486 (Omer Asik) has been added
Player 22 of 486 (D.J. Augustin) has been added
Player 23 of 486 (Luke Babbitt) has been added
Player 24 of 486 (Ron Baker) has been a

Player 174 of 486 (Justin Harper) has been added
Player 175 of 486 (Montrezl Harrell) has been added
Player 176 of 486 (Devin Harris) has been added
Player 177 of 486 (Gary Harris) has been added
Player 178 of 486 (Joe Harris) has been added
Player 179 of 486 (Manny Harris) has been added
Player 180 of 486 (Tobias Harris) has been added
Player 181 of 486 (Aaron Harrison) has been added
Player 182 of 486 (Andrew Harrison) has been added
Player 183 of 486 (Udonis Haslem) has been added
Player 184 of 486 (Spencer Hawes) has been added
Player 185 of 486 (Gordon Hayward) has been added
Player 186 of 486 (Gerald Henderson) has been added
Player 187 of 486 (John Henson) has been added
Player 188 of 486 (Juan Hernangomez) has been added
Player 189 of 486 (Willy Hernangomez) has been added
Player 190 of 486 (Mario Hezonja) has been added
Player 191 of 486 (Roy Hibbert) has been added
Player 192 of 486 (Buddy Hield) has been added
Player 193 of 486 (Nene Hilario) has been added
Player 194 of 486

Player 342 of 486 (Chinanu Onuaku) has been added
Player 343 of 486 (Kelly Oubre) has been added
Player 344 of 486 (Zaza Pachulia) has been added
Player 345 of 486 (Georgios Papagiannis) has been added
Player 346 of 486 (Jabari Parker) has been added
Player 347 of 486 (Tony Parker) has been added
Player 348 of 486 (Chandler Parsons) has been added
Player 349 of 486 (Lamar Patterson) has been added
Player 350 of 486 (Patrick Patterson) has been added
Player 351 of 486 (Chris Paul) has been added
Player 352 of 486 (Adreian Payne) has been added
Player 353 of 486 (Cameron Payne) has been added
Player 354 of 486 (Elfrid Payton) has been added
Player 355 of 486 (Gary Payton) has been added
Player 356 of 486 (Paul Pierce) has been added
Player 357 of 486 (Marshall Plumlee) has been added
Player 358 of 486 (Mason Plumlee) has been added
Player 359 of 486 (Miles Plumlee) has been added
Player 360 of 486 (Jakob Poeltl) has been added
Player 361 of 486 (Otto Porter) has been added
Player 362 of 

## Combine into One Dataframe and Save

In [48]:
df = pd.concat(merged_list)
df.shape

(26138, 281)

In [49]:
df.to_csv('./Data/2016-2017season.csv')